<a href="https://colab.research.google.com/github/fubotz/ICL_2024W/blob/main/FinalProject_Fabian_SCHAMBECK_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Project: Finetuning a Multilingual Model
Description here...

Model:

Dataset:

In [1]:
!pip install bertviz
!pip install datasets
!pip install evaluate
!pip install transformers
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beha

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch

# Load the multilingual DistilBERT model and tokenizer
model_name = "distilbert-base-multilingual-cased"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

print(tokenizer)

DistilBertTokenizerFast(name_or_path='distilbert-base-multilingual-cased', vocab_size=119547, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)


In [20]:
# Define hardcoded word pairs for evaluation (n=21)
word_pairs = [
    ("academic", "académique"),
    ("administrator", "administrateur"),
    ("algorithm", "algorithme"),
    ("chemical", "chimique"),
    ("delicious", "délicieux"),
    ("emotion", "émotion"),
    ("exercise", "exercice"),
    ("gender", "genre"),
    ("gorilla", "gorille"),
    ("loyalty", "loyauté"),
    ("notation", "notamment"),
    ("objective", "objectif"),
    ("oratory", "oratoire"),
    ("particle", "particule"),
    ("quarter", "quartier"),
    ("september", "septembre"),
    ("skeleton", "squelette"),
    ("traditionally", "traditionnellement"),
    ("voice", "voix"),
    ("west", "ouest"),
    ("wine", "vin"),
]

In [21]:
def get_embedding_for_word(word):
    """
    Computes the embedding for a given word dynamically.

    Parameters:
        word (str): The input word.

    Returns:
        np.ndarray: The embedding of the input word as a numpy array.
    """
    # Tokenize the input word
    tokens = tokenizer(word, return_tensors="pt", padding=True, truncation=True)

    # Compute the embedding dynamically using the model
    with torch.no_grad():
        outputs = model(**tokens)
    hidden_states = outputs.last_hidden_state

    # Pool embeddings for the input word
    embedding = hidden_states[0, 1:-1].mean(dim=0)  # Average token embeddings
    return embedding.numpy()


In [22]:
def get_embedding_for_word(word):
    """
    Computes the embedding for a given word dynamically.

    Parameters:
        word (str): The input word.

    Returns:
        np.ndarray: The embedding of the input word as a numpy array.
    """
    tokens = tokenizer(word, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokens)
    hidden_states = outputs.last_hidden_state
    embedding = hidden_states[0, 1:-1].mean(dim=0)  # Average token embeddings
    return embedding.numpy()

In [23]:
def precompute_embeddings(words, model):
    """
    Precomputes embeddings for a list of words.

    Parameters:
        words (list of str): The list of words to compute embeddings for.
        model (AutoModel): The pretrained model.

    Returns:
        dict: A dictionary mapping words to their embeddings.
    """
    embeddings = {}
    for word in words:
        embeddings[word] = get_embedding_for_word(word)
    return embeddings

# Extract French words from word_pairs and precompute their embeddings
french_words = [pair[1] for pair in word_pairs]
target_embeddings = precompute_embeddings(french_words, model)

print(f"Precomputed embeddings for {len(target_embeddings)} French words.")


Precomputed embeddings for 21 French words.


In [24]:
def get_nn(word, target_embeddings, top_k=5):
    """
    Finds the nearest neighbors for a given word using cosine similarity.

    Parameters:
        word (str): The input word.
        target_embeddings (dict): Precomputed embeddings for the target words.
        top_k (int): Number of top neighbors to return.

    Returns:
        list of tuple: The top-k most similar words and their similarity scores.
    """
    # Compute the embedding for the input word
    input_embedding = get_embedding_for_word(word)
    input_embedding = input_embedding / np.linalg.norm(input_embedding)  # Normalize

    # Extract embeddings and words from the target set
    target_words = list(target_embeddings.keys())
    target_vecs = np.array(list(target_embeddings.values()))

    # Normalize target embeddings
    target_vecs_norm = target_vecs / np.linalg.norm(target_vecs, axis=1, keepdims=True)

    # Compute cosine similarities
    similarities = cosine_similarity([input_embedding], target_vecs_norm).flatten()

    # Find top-k most similar words
    nearest_idxs = similarities.argsort()[-top_k:][::-1]
    return [(target_words[i], similarities[i]) for i in nearest_idxs]

# Example: Find nearest neighbors for an English word
input_word = "academic"
nearest_neighbors = get_nn(input_word, target_embeddings, top_k=5)
print(f"Nearest neighbors for '{input_word}': {nearest_neighbors}")


Nearest neighbors for 'academic': [('exercice', 0.5766833), ('objectif', 0.5698213), ('notamment', 0.5437145), ('émotion', 0.49054876), ('vin', 0.48356932)]


In [25]:
def evaluate_cognate_detection(word_pairs, target_embeddings, top_k=5):
    """
    Evaluates the model's ability to predict French cognates for English words.

    Parameters:
        word_pairs (list of tuple): A list of (input_word, expected_word) pairs.
        target_embeddings (dict): Precomputed embeddings for the target words.
        top_k (int): Number of top neighbors to consider.

    Returns:
        list of dict: A list of results for each test case, including input, expected, and found words.
    """
    results = []
    for input_word, expected_word in word_pairs:
        # Find the nearest neighbors
        nearest_neighbors = get_nn(input_word, target_embeddings, top_k=top_k)

        # Get the best match
        found_word, similarity = nearest_neighbors[0]

        # Append results
        results.append({
            "input_word": input_word,
            "expected_word": expected_word,
            "found_word": found_word,
            "similarity": similarity,
            "top_k": nearest_neighbors
        })
    return results

# Run the evaluation
results = evaluate_cognate_detection(word_pairs, target_embeddings, top_k=5)

# Display the results
for result in results:
    print(f"Input: {result['input_word']}, Expected: {result['expected_word']}, Found: {result['found_word']}, Similarity: {result['similarity']:.4f}")


Input: academic, Expected: académique, Found: exercice, Similarity: 0.5767
Input: administrator, Expected: administrateur, Found: exercice, Similarity: 0.6804
Input: algorithm, Expected: algorithme, Found: algorithme, Similarity: 0.7911
Input: chemical, Expected: chimique, Found: objectif, Similarity: 0.6137
Input: delicious, Expected: délicieux, Found: émotion, Similarity: 0.5190
Input: emotion, Expected: émotion, Found: émotion, Similarity: 0.5554
Input: exercise, Expected: exercice, Found: exercice, Similarity: 0.7389
Input: gender, Expected: genre, Found: objectif, Similarity: 0.5006
Input: gorilla, Expected: gorille, Found: gorille, Similarity: 0.7423
Input: loyalty, Expected: loyauté, Found: loyauté, Similarity: 0.5873
Input: notation, Expected: notamment, Found: objectif, Similarity: 0.6474
Input: objective, Expected: objectif, Found: objectif, Similarity: 0.8511
Input: oratory, Expected: oratoire, Found: oratoire, Similarity: 0.8114
Input: particle, Expected: particule, Found: 